<b>1st Part</b>

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
# from sklearn.cluster import KMeans 
# import folium 

import requests

#do not need to install these every time
!pip install bs4
!pip install lxml

import lxml
from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


In [5]:
req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

soup = BeautifulSoup(req.content, 'lxml')
table = soup.find_all('table')[0]

df = pd.read_html(str(table))
neighborhood=pd.DataFrame(df[0])
#remove rows where 'Borough' equals 'Not assigned'
neighborhood.drop(neighborhood.index[neighborhood['Borough'] == 'Not assigned'], inplace = True)

#combining rows with same Postcodes
neighborhood = neighborhood.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
#print(neighborhood)

neighborhood.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [4]:
#number of rows in the dataframe
neighborhood.shape[0]

103

<b>2nd part</b>

In [6]:
geospatial_data = pd.read_csv('https://cocl.us/Geospatial_data')
#print(geospatial_data)

geospatial_data.rename(columns={'Postal Code':'Postcode'}, inplace=True)

#new datafram with geospatial_data
neib_with_geo = pd.merge(neighborhood, geospatial_data, on='Postcode')

neib_with_geo.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


<b>3rd part</b>

In [ ]:
#datafram where all Neighbourhoods contain 'Toronto'
toronto_data = neib_with_geo[neib_with_geo['Borough'].str.contains("Toronto")].reset_index(drop=True)
toronto_data.head()
toronto_data.shape[0]

!pip install folium
import folium

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Solving environment: / 